In [1]:
import sys
from dotenv import load_dotenv
import pandas as pd
sys.path.insert(1, '/home/jabez/week_11/Contract-Advisor-RAG')
load_dotenv()
sys.path.insert(1, '/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/scripts')
import file_loader 
import pipelines 
import evaluation

/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# loading data
file_path = '/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/data/cnn_dailymail_3.0.0.csv'
data = file_loader.load_csv(file_path)

In [4]:
# RecursiveCharacterTextSplitter
vectorstore_character = file_loader.character_text_splitter(data)

In [8]:
# Generate syntetic test data
file_path = '/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/test_data/syntetic_test_data.csv'
syntetic_test_data =evaluation.generate_syntetic_testdata(data, file_path)

Filename and doc_id are the same for all nodes.                     
Generating: 100%|██████████| 10/10 [00:42<00:00,  4.22s/it]


In [22]:
syntetic_test_data = pd.read_csv('/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/test_data/syntetic_test_data.csv')

In [9]:
# Setting retriever
retriver = vectorstore_character.as_retriever(search_type="similarity", search_kwargs={"k": 6})


In [10]:
# Adding answer to test data from simple pipeline
file_path_with_answer = '/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/test_data/syntetic_test_data_with_answer.csv'
syntetic_test_data_with_answer = evaluation.adding_answer_to_testdata(syntetic_test_data, pipelines.simple_pipeline, vectorstore_character, retriver, file_path_with_answer)

/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 84.74ba/s]


In [12]:
# Evaluating the test data from simple pipeline
simple_rag_evaluation_result = evaluation.ragas_evaluator(syntetic_test_data_with_answer)

Evaluating: 100%|██████████| 40/40 [00:18<00:00,  2.21it/s]


In [13]:
# Evaluation mean
def evaluation_mean(evaluation_result):
    context_precision = round(evaluation_result['context_precision'].mean()*100,2)
    faithfulness = round(evaluation_result['faithfulness'].mean()*100,2)
    answer_relevancy = round(evaluation_result['answer_relevancy'].mean()*100,2)
    context_recall = round(evaluation_result['context_recall'].mean()*100,2)
    print(f'context_precision: {context_precision}%, faithfulness: {faithfulness}%, answer_relevancy: {answer_relevancy}%, context_recall: {context_recall}%')

result = evaluation_mean(simple_rag_evaluation_result)

context_precision: 86.5%, faithfulness: 83.98%, answer_relevancy: 86.76%, context_recall: 81.67%


In [14]:
simple_rag_evaluation_result

,question,answer,contexts,ground_truth,context_precision,faithfulness,answer_relevancy,context_recall
0,What group claimed responsibility for the host...,The left-wing Revolutionary People's Liberatio...,[siege began. Istanbul Police Chief Selami Alt...,The left-wing Revolutionary People's Liberatio...,0.950000,1.000000,0.976869,1.000000
1,How does the perception of authenticity impact...,The perception of authenticity impacts the val...,[puzzling out how to insert themselves into a ...,The perception of authenticity can impact the ...,1.000000,1.000000,0.928415,0.666667
2,What was the cause of Conrad Roy's death?,Conrad Roy's cause of death was apparent carbo...,"[highlights: Conrad Roy, 18, was found dead of...",Conrad Roy's cause of death was apparent carbo...,0.833333,1.000000,0.990655,1.000000
3,What are the pros and cons of the Julia Robert...,The pros and cons of the Julia Roberts-Richard...,[article: (CNN)For a generation of moviegoers ...,Roberts (as Vivian) is empowered and shows hum...,1.000000,0.842105,0.999146,1.000000
4,How does the use of the n-word impact the fina...,The information is not available in the provid...,"[article: (CNN)Things got testy on ""CNN Tonigh...",The use of the n-word does not have a signific...,0.950000,0.000000,0.000000,0.666667
5,How has the email issue affected Hillary Clint...,The email issue has affected Hillary Clinton's...,"[is actually running, with an infrastructure b...",The email issue has affected Hillary Clinton's...,1.000000,0.888889,0.945375,0.833333
6,Where will the first U.S. show of the Hit & Ru...,The first U.S. show of the Hit & Run Tour will...,[article: (CNN)Following last year's successfu...,"Louisville, Kentucky",0.916667,1.000000,0.934299,1.000000
7,What is the starting bid for the Grantville bu...,The starting bid for the Grantville buildings ...,[article: (CNN)The faded downtown of a small t...,The starting bid for the Grantville buildings ...,1.000000,1.000000,0.963829,1.000000
8,What is Kick It Out's role in addressing racis...,Kick It Out (KIO) plays a significant role in ...,[article: (CNN)Could mobile technology turn th...,Kick It Out is an anti-racism organization tha...,1.000000,1.000000,0.937225,1.000000
9,What is the significance of the Jessup facilit...,The significance of the Jessup facility in rel...,"[not constitute prosecutable violations"" of fe...",The answer to given question is not present in...,0.000000,0.666667,1.000000,0.000000


In [29]:
simple_rag_evaluation_result.to_csv('/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/test_data/simple_rag_evaluation_result.csv', index=False)